In [ ]:
#pip install pandas


In [ ]:
#pip install matplotlib

In [ ]:
pip install openpyxl

In [ ]:
import xpress as xp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Create a problem 

stage_1 = xp.problem(name='F_O stage 1')

In [ ]:
# Import data into panda dataframes

df_rooms = pd.read_excel('Relevant Data Room.xlsx', header=0)
df_course_demand = pd.read_excel('Course Data.xlsx', header=0, sheet_name = 'Dict')[['Size']]
df_courses = pd.read_excel('Course Data.xlsx', header=0, sheet_name='Dict')
df_curricula = pd.read_excel('Course Data.xlsx', header=0, sheet_name='Programs')

df_rooms_types = pd.read_excel('Relevant Data Room.xlsx', header=0, sheet_name='ROOM_DIC')[['ROOM INDEX', 'ROOM NAME', 'CAP', 'ROOM LAYOUT']]
#df_courses

In [ ]:
cu_1 = list(df_curricula['Y1'])
cu_2 = list(df_curricula['Y2'])
cu_3 = list(df_curricula['Y3'])
df_curricula

In [ ]:
course_code = df_courses['Course Code'].to_numpy()
course_code

In [ ]:
df_curricula

In [ ]:
cu_1 = list(df_curricula['Y1'])
cu_2 = list(df_curricula['Y2'])
cu_3 = list(df_curricula['Y3'])

c = cu_1[7]
c = c.split(',')
i = np.where(np.isin(course_code, np.array(c)))
print(c)
print(i)
print(course_code[23])

In [ ]:
# Index sets
courses = list(df_courses['Course'])
course_code = df_courses['Course Code'].to_numpy()
demand = list(df_course_demand['Size'])
curricula = list(df_curricula['Code'])
timeslots = [i for i in range(9*5)] 
print(timeslots)
days = ['M_O', 'T_O', 'W_O', 'TH_O', 'F_O'] #Days for odd week
rooms = list(df_rooms['ROOM INDEX'])
room_type = [1,2,3]
years = [1,2,3,4,5,'PGT']
#Lectures can happen in Theatre, Classroom
#Workshops happen in Teaching Studio, Classroom, Boardroom 
class_type = ['L', 'W']
workshop_pattern = [1, 2, 3]

capacities = sorted(np.append(df_rooms['CAP'].unique(), 0))
# Parameters 
lecture_hours = list(df_courses['Lecture - Duration'])
workshop_hours = list(df_courses['Workshop - Duration'])



# Create the variables
x = np.array([xp.var( name='x_{0}_{1}_{2}'.format(i, j, k), vartype=xp.binary)
                  for i in courses for j in timeslots for k in class_type], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(class_type))

y = np.array([xp.var( name='y_{0}_{1}_{2}_{3}'.format(i, j, s, k), vartype=xp.binary)
                  for i in courses for j in timeslots for s in capacities for k in class_type], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(capacities), len(class_type))

z = np.array([xp.var( name='z_{0}_{1}'.format(i, d), vartype=xp.binary)
                  for i in courses for d in days], dtype=xp.npvar).reshape(len(courses), len(days))

w = np.array([xp.var( name='w_{0}'.format(i))
                    for i in courses], dtype=xp.npvar)

r = np.array([xp.var( name='r_{0}_{1}_{2}'.format(cu, j, k), vartype=xp.binary)
                  for  cu in curricula for j in timeslots for k in range(3)], dtype=xp.npvar).reshape(len(curricula), len(timeslots), 3)

v = np.array([xp.var( name='v_{0}_{1}'.format(cu, j), vartype=xp.binary)
                  for cu in curricula for j in timeslots], dtype=xp.npvar).reshape(len(curricula), len(timeslots))

#counts number of clashes 
h = np.array([xp.var( name='h_{0}_{1}'.format(i, j))
                    for i in years for j in timeslots], dtype=xp.npvar).reshape(len(years), len(timeslots))





In [ ]:
# Add the variables to the problem
stage_1.addVariable(x, y, z, w, r, v, h)



In [ ]:
print(timeslots)


In [ ]:
#Add constraints

#C1: A course i in C should be assigned exactly L(c) lectures that is L(c)== #timeslots(c)
C1_L = [xp.Sum(x[i, j, 0] for j in (timeslots)) == lecture_hours[i] for i in range(len(courses))]
#for workshops
C1_W = [xp.Sum(x[i, j, 1] for j in (timeslots)) == workshop_hours[i] for i in range(len(courses))]

#C2: At s given time period we do not assign more courses than available rooms in general for both types

#!!! separate the types
C2 = [xp.Sum(x[i, j, k] for i in range(len(courses)) for k in range(len(class_type))) <= len(rooms) for j in (timeslots)]

# Lectures on types 1, 3
#C2_L = [x[i, j, 0] == 0 for i in range(len(courses)) for j in timeslots]
# Workshops on types 2, 3
#C2_W = [x[i, j, 0] == 0 for i in range(len(courses)) for j in timeslots]

#C3: Makes sure that if a room is assigned to a course in a time period, then course is assigned to that time period
C3 = [x[i,j,k] - y[i,j,s,k] >= 0 for i in range(len(courses)) for j in (timeslots) for k in range(len(class_type)) for s in range(len(capacities))]

#C4: ?

#C5: Calculates if classtypes from a course are planned on a day 
C5 = [xp.Sum(x[i,j,k] for j in (timeslots)) - z[i,d] >= 0 for i in range(len(courses)) for k in (range(len(class_type))) for d in range(len((days)))]

#C6: Violation of mnd(c)
#C6 = 

#C7: Only one course from curriculum is planned at timeslot t
#NEED COURSES IN CURRICULA FOR THIS 
# REMEMBER TO CHANGE THISSSSS
cu_1 = list(df_curricula['Y1'])
cu_2 = list(df_curricula['Y2'])
cu_3 = list(df_curricula['Y3'])


C7_1 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, np.array(cu_1[cu].split(', '))))[0] for k in range(len(class_type))) -r[cu, j, 0] == 0 for cu in range(len(curricula)) for j in (timeslots)]
C7_2 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, np.array(cu_2[cu].split(', '))))[0] for k in range(len(class_type))) -r[cu, j, 1] == 0 for cu in range(len(curricula)) for j in (timeslots)]
C7_3 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, np.array(cu_3[cu].split(', '))))[0] for k in range(len(class_type))) -r[cu, j, 2] == 0 for cu in range(len(curricula)) for j in (timeslots)]

#C8: Calculates curriculum compactness violation 
#C8 = [-r[cu,j-1] + r[cu,j] - r[cu, j+1] - v[cu,j] <= 0 for cu in range(len(curricula)) for j in [1,2,3,4,5,6,7]]

#C9: Demand is satisfied for classes

C9 = [y[i, j, s, k] == 0 for i in range(len(courses)) for j in timeslots for k in range(len(class_type)) for s in range(len(capacities)) if s < demand[i]]



df_courses_year = pd.read_excel('Course Data.xlsx', header=0, sheet_name='CoursesYears')
year1_courses = df_courses_year['Y1'].dropna().to_numpy()
year2_courses = df_courses_year['Y2'].dropna().to_numpy()
year3_courses = df_courses_year['Y3'].dropna().to_numpy()
year4_courses = df_courses_year['Y4'].dropna().to_numpy()
year5_courses = df_courses_year['Y5'].dropna().to_numpy()
pgt_courses = df_courses_year['PGT'].dropna().to_numpy()


#C10 
C10_1 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, year1_courses))[0] for k in range(len(class_type))) <= h[0, j] for j in (timeslots)]
C10_2 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, year2_courses))[0] for k in range(len(class_type))) <= h[1, j] for j in (timeslots)]
C10_3 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, year3_courses))[0] for k in range(len(class_type))) <= h[2, j] for j in (timeslots)]
C10_4 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, year4_courses))[0] for k in range(len(class_type))) <= h[3, j] for j in (timeslots)]
C10_5 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, year5_courses))[0] for k in range(len(class_type))) <= h[4, j] for j in (timeslots)]
C10_PGT = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, pgt_courses))[0] for k in range(len(class_type))) <= h[5, j] for j in (timeslots)]


In [ ]:
#Add constraints 
#stage_1.addConstraint(C1_L, C1_W, C2, C3, C5, C7_1, C7_2, C7_3, C9, C10_1, C10_2, C10_3, C10_4, C10_5, C10_PGT)
stage_1.addConstraint(C1_L, C1_W, C2, C3, C5, C7_1, C7_2, C7_3, C10_1, C10_2, C10_3, C10_4, C10_5, C10_PGT)

In [ ]:
#Write objective function 
stage_1.setObjective(xp.Sum(h[i,j] for i in range(len(years)) for j in range(len(timeslots))), sense = xp.minimize)

In [ ]:
stage_1.getConstraint()

In [ ]:
stage_1.solve()

In [ ]:
#make_df = pd.DataFrame(data = stage_1.getSolution(), index = course_code, columns = timeslots).to_numpy()
make_df = pd.DataFrame(data = stage_1.getSolution(x)).to_numpy()
print(make_df)
make_df = make_df.reshape(len(courses), len(timeslots), len(class_type))
make_df

In [ ]:
# Create a problem 

stage_2 = xp.problem(name='F_O stage 2')

In [ ]:
# Create the variables

#u = np.array([xp.var( name='u_{0}_{1}_{2}_{3}'.format(i, j, k, l), vartype=xp.binary)
#                  for i in courses for j in timeslots for k in rooms for l in class_type], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(rooms), len(class_type))
u = np.array([xp.var( name='u_{0}_{1}_{2}'.format(i, j, k), vartype=xp.binary)
                  for i in courses for j in timeslots for k in rooms], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(rooms))

t = np.array([xp.var( name='t_{0}_{1}'.format(i, j), vartype=xp.binary)
                  for i in courses for j in rooms], dtype=xp.npvar).reshape(len(courses), len(rooms))



In [ ]:
# Add the variables to the problem
stage_2.addVariable(u, t)

In [ ]:
# Constraints
# Remeber to change c_2 in accordance to solution from stage 1

#b_2 = [xp.Sum(u[i, j, k, l] for j in timeslots) - len(timeslots) * t[i, k] <= 0 for i in range(len(courses)) for k in range(len(rooms)) for l in range(len(class_type))]
#c_2 = [xp.Sum(u[i, j, k, l] for k in range(len(rooms))) == 1 for i in range(len(courses)) for j in timeslots for l in range(len(class_type))]
#d_2 = [xp.Sum(u[i, j, k, l] for i in range(len(courses)) for j in timeslots) <= 1 for k in  range(len(rooms)) for l in range(len(class_type))]
b_2 = [xp.Sum(u[i, j, k] for j in timeslots) - len(timeslots) * t[i, k] <= 0 for i in range(len(courses)) for k in range(len(rooms))]
c_2 = [xp.Sum(u[i, j, k] for k in range(len(rooms))) == 1 for i in range(len(courses)) for j in timeslots]
d_2 = [xp.Sum(u[i, j, k] for i in range(len(courses)) for j in timeslots) <= 1 for k in  range(len(rooms))]


In [ ]:
stage_2.addConstraint(b_2, c_2, d_2)

In [ ]:
stage_2.setObjective(xp.Sum(t[i, j] for i in range(len(courses)) for j in range(len(rooms))), sense = xp.minimize)
stage_2.solve()